In [1]:
import wave
import numpy as np
import python_speech_features as ps
import os
import glob
import pickle

epsilon = 1e-5

def wgn(x, snr):
    snr = 10**(snr/10.0)
    xpower = np.sum(x**2)/len(x)
    npower = xpower / snr
    return np.random.randn(len(x)) * np.sqrt(npower)

def read_file(filename):
    file = wave.open(filename,'r')    
    params = file.getparams()
    nchannels, sampwidth, framerate, wav_length = params[:4]
    str_data = file.readframes(wav_length)
    wavedata = np.fromstring(str_data, dtype = np.short)
    # librosa.load(wav_file_path + orig_wav_file, sr=sr)
    time = np.arange(0,wav_length) * (1.0/framerate)
    file.close()
    return wavedata, time, framerate

def generate_label(control):
    label = 0
    if(control == 'cc'):
        label = 0
    elif(control == 'cd'):
        label = 1
    return label


In [2]:
filter_num = 128
t_ = 512
rootdir = 'train/Full_wave_enhanced_audio'

train_label = []
train_data = []

In [3]:
# for control in os.listdir(rootdir):
    
#     sub_dir = rootdir + '/' + control
    
#     for sample in os.listdir(sub_dir):
#         data, time, rate = read_file(sub_dir+'/'+sample)
#         mel_spec = ps.logfbank(data,rate,nfilt = filter_num)
        
# #         print(data.shape[0])
#         time = mel_spec.shape[0] 
# #         print(mel_spec.shape)
# #         print(delta1.shape)
# #         print(delta2.shape)
        
#         length = time//256 * 257
        
#         print(length )
#         mel_spec = mel_spec[:length]
#         print(mel_spec.shape)
# #         delta1 = ps.delta(mel_spec, 2)
# #         delta2 = ps.delta(delta1, 2)
# #         print(time//256)
# #         for i in range(time//256):
# #             begin = 256*i
# #             end = begin + 256
# # #             if (time-begin<256): 
# # #                 end = time
# # #                 part = pad(mel_spec[begin:end,:],256,((0, 256),(before_N, after_N)),mode='constant')
# # #                 delta11 = delta1[begin:end,:]
# # #                 delta21 = delta2[begin:end,:]
                
                
# # #             else:
# #             part = mel_spec[begin:end,:]
# #             delta11 = delta1[begin:end,:]
# #             delta21 = delta2[begin:end,:]
                
            
            
# #             train_label.append(generate_label(control))
#         break  
          

In [4]:
# train_label

In [5]:
for control in os.listdir(rootdir):
    
    sub_dir = rootdir + '/' + control
    
    for sample in os.listdir(sub_dir):
        data, time, rate = read_file(sub_dir+'/'+sample)
        mel_spec = ps.logfbank(data,rate,nfilt = filter_num)
        time = mel_spec.shape[0] 
        mel_spec = mel_spec[:time//t_ *t_,:]
        time = mel_spec.shape[0] 
        delta1 = ps.delta(mel_spec, 2)
        delta2 = ps.delta(delta1, 2)
        for i in range(time//t_):
            begin = t_*i
            end = begin + t_
            
            part = mel_spec[begin:end,:]
            delta11 = delta1[begin:end,:]
            delta21 = delta2[begin:end,:]
            
            _data = np.empty((3,t_,filter_num),dtype = np.float32)
            
            _data[0,:,:] = part
            _data[1,:,:] = delta11
            _data[2,:,:] = delta21
            
            train_label.append(generate_label(control))
            train_data.append(_data)

#         print(mel_spec[:time//256 *256,:].shape)
#         break
#     break


C:\Users\Asus\AppData\Local\Temp/ipykernel_21360/3645145104.py:21: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  wavedata = np.fromstring(str_data, dtype = np.short)


In [6]:
np.array(train_data).shape

(1466, 3, 512, 128)

In [7]:
np.array(train_label).shape

(1466,)

In [8]:
from sklearn.model_selection import train_test_split
train_size=0.8

X_train, X_valid, y_train, y_valid = train_test_split(np.array(train_data),np.array(train_label), train_size=0.95)

In [9]:
np.shape(X_train),np.shape(X_valid)

((1392, 3, 512, 128), (74, 3, 512, 128))

In [10]:
output = './adress_Att_Net.pkl'
f=open(output,'wb') 
pickle.dump((X_train,y_train,X_valid,y_valid),f)
f.close() 